In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
spark

In [5]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
 

In [6]:
#get and store the data

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    rdd.coalesce(1).saveAsTextFile('file:///C:/Users/lenne/Desktop/spark/coding_and_data/data/new_data')
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()

In [11]:
ssc = StreamingContext(sc, 2000)

In [12]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [13]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2021-04-22 10:26:40 =========


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\lenne\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-19fbf29765bb>", line 9, in run
    ssc.awaitTermination()
  File "C:\Users\lenne\Desktop\spark\spark-3.1.1-bin-hadoop2.7\python\pyspark\streaming\context.py", line 199, in awaitTermination
    self._jssc.awaitTermination()
  File "C:\Users\lenne\Desktop\spark\spark-3.1.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1305, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "C:\Users\lenne\Desktop\spark\spark-3.1.1-bin-hadoop2.7\python\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "C:\Users\lenne\Desktop\spark\spark-3.1.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: An error occurred while calling o155.await

In [ ]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
